# Preparing Data for Neuralangelo

Author: Deeepwin  
Date: 08.07.2023 
***

### Literature

Related Literature:  

- [Nerualangelo Supplementary](https://research.nvidia.com/labs/dir/neuralangelo/supplementary.pdf) containing details information on results.
- [InstantNGP](https://github.com/NVlabs/instant-ngp/blob/master/docs/nerf_dataset_tips.md#COLMAP) instructions to prepare new custom image datasets using COLMAP.  

  The fox example dataset can be found in original [InstantNGP repository](https://github.com/NVlabs/instant-ngp/tree/master/data/nerf/fox). InstantNGP implementation expects initial camera parameters to be provided in a `transforms.json` file in a format compatible with [the original NeRF codebase](https://www.matthewtancik.com/nerf) contains the pose information for each image.  

  The script `colmap2nerf.py` can be used to convert a text colmap export `images.txt` to nerf format `transforms.json`. However, script require as well the COLMAP `cameras.txt` containing the intrinsic parameters of all cameras.

- [NeRF Original Paper](https://github.com/bmild/nerf#already-have-poses) showing how to load already existing poses.

- [COLMAP Output Format](https://colmap.github.io/format.html#images-txt) explains the `images.txt` which contains the poses and keypoints of all images.



### Datasets

Benchmarks for 3D Surface Reconstruction used by NeuralAngelo:

- [Tanks and Temples](https://www.tanksandtemples.org/download/): Input is images or video. Poses are calculated from reconstruction using COLMAP into a [.log](http://redwood-data.org/indoor/fileformat.html) file format.

### Installation

Run this command in a terminal to create new conda environment:     

`conda create --name sdfstudio python=3.8`

In [ ]:
# install pytorch with CUDA
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# requires nvcc
!conda install -y -c "nvidia/label/cuda-11.3.1" cuda-toolkit
!conda install -y -c "nvidia/label/cuda-11.3.1" cuda-nvcc

In [ ]:
# set path to CUDA installation
!conda env config vars set CUDA_HOME="/home/martin/anaconda3/envs/sdfstudio/"
!conda env config vars list

In [ ]:
# check CUDA versions, must be 11.3
!conda list cuda-nvcc

In [ ]:
# require version 9, change default version 'sudo -S update-alternatives --config gcc'
from getpass import getpass
p = getpass()
!echo {p} | sudo -S apt -y install g++-9
!echo {p} | sudo -S update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-9 9
!echo {p} | sudo -S update-alternatives --config g++
!echo {p} | sudo -S apt -y install gcc-9
!echo {p} | sudo -S update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 9
!echo {p} | sudo -S update-alternatives --config gcc
del p

In [ ]:
# install tinycudann (you might need to run this command in separate terminal in case of an error)
!pip install ninja git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

In [ ]:
# install nerfstudio as well
!pip install nerfstudio

In [ ]:
# install SDF studio
!git clone https://github.com/autonomousvision/sdfstudio.git
%cd sdfstudio
!pip install -e .
%cd ..

In [ ]:
# reinstall to ensure that torch and CUDA have same version, as previous installation steps might have changed it
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# nerf studio utilities
!ns-install-cli

#### Test SDF Studio

In [ ]:
!ns-download-data sdfstudio

In [ ]:
# train model from scratch on the dtu dataset scan65, default iteration is 20000
!ns-train neus-facto --trainer.max-num-iterations 100 --pipeline.model.sdf-field.inside-outside False --vis viewer --experiment-name neus-facto-dtu65 sdfstudio-data --data data/sdfstudio-demo-data/dtu-scan65

In [ ]:
# copy paste output directory
output_dir = 'outputs/neus-facto-dtu65/neus-facto/2023-07-28_094017'

In [ ]:
# extract mesh
!ns-extract-mesh --load-config $output_dir/config.yml --output-path exports/neus-facto-dtu65/mesh.ply

In [ ]:
# export as point cloud, adjust export directory
!ns-export pointcloud --load-config $output_dir/config.yml --output-dir exports/neus-facto-dtu65/ --num-points 1000 --remove-outliers True --use-bounding-box True --bounding-box-min -1 -1 -1 --bounding-box-max 1 1 1

In [ ]:
# look at mesh
!open3d draw exports/neus-facto-dtu65/mesh.ply

#### Training with Neuralangelo

In [ ]:
# define custom data folder location
input_dir = 'data/mine/mouse-2'

In [ ]:
# show list of available models
!ns-train --help

In [ ]:
# convert nerfstudio (COLMAP) to sdfstudio fromat, check if conversion is correct
!python scripts/datasets/process_nerfstudio_to_sdfstudio.py --data $input_dir --output-dir $input_dir-ns  --data-type colmap --scene-type object

For different model options see here: [Models](https://vscode.dev/github/autonomousvision/sdfstudio/blob/master/nerfstudio/configs/method_configs.py#L107). Relevant options for me are:

- "neuralangelo": "Implementation of Neuralangelo"
- "bakedangelo": "Implementation of Neuralangelo with BakedSDF"
- "neus-facto-angelo": "Implementation of Neuralangelo with neus-facto"
- "instant-ngp": "Implementation of Instant-NGP. Recommended real-time model for bounded synthetic data."
- "mipnerf": "High quality model for bounded scenes. (slow)"
    

In [ ]:
# train, use nerfstudio data fromat directly with option `nerfstudio-data`
!ns-train neuralangelo --pipeline.model.sdf-field.inside-outside False nerfstudio-data --data $input_dir